In [1]:
!which python

/home/bingyu/anaconda3/envs/py38/bin/python


In [2]:
from keplergl import KeplerGl
# tmp = map1.config
# with open('keplergl_config.json', 'w') as outfile:
#     json.dump(tmp, outfile, indent=2)
# map_config = json.load(open('keplergl_config.json'))
# map1=KeplerGl(height=400, config=map_config)
# map1.config = map_config
# map1
# map1.save_to_html(file_name="my_keplergl_map.html")

import model.dta_meso_butte as dta_meso

import json
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point
from shapely.wkt import loads

import random
import numpy as np
random.seed(0)
np.random.seed(0)

In [3]:
def extract_vehicle_locations(network):
    agents_dict = network.agents.copy()
    links_dict = network.links.copy()
    queue_vehicle_position = []
    run_vehicle_position = []
    closed_link_position = []
    for link_id, link in links_dict.items():
        if link.link_type == 'v':
            continue
        if link.burnt == 'burnt':
            closed_link_position.append([link.link_id, link.midpoint[0], link.midpoint[1]])
        link_geometry = link.geometry
        link_length = link.geometry.length
        link_lanes = link.lanes
        queue_vehicle_i, run_vehicle_i = 0, 0
        for vehicle in link.queue_vehicles:
            vehicle_pos = link_geometry.interpolate( link_length - queue_vehicle_i*8/link_lanes )
            queue_vehicle_position.append([vehicle, vehicle_pos.x, vehicle_pos.y])
            queue_vehicle_i += 1
        for vehicle in link.run_vehicles:
            vehicle_pos = link_geometry.interpolate( run_vehicle_i*8/link_lanes )
            run_vehicle_position.append([vehicle, vehicle_pos.x, vehicle_pos.y])
            run_vehicle_i += 1
    return run_vehicle_position, queue_vehicle_position, closed_link_position

def make_gdf(vehicle_position_list):
    vehicle_position_df = pd.DataFrame(vehicle_position_list, columns=['vehicle_id', 'x', 'y'])
    vehicle_position_gdf = gpd.GeoDataFrame(vehicle_position_df, crs='epsg:26910', geometry=[Point(xy) for xy in zip(vehicle_position_df['x'], vehicle_position_df['y'])]).to_crs('epsg:4326')
    vehicle_position_gdf['lon'] = vehicle_position_gdf['geometry'].apply(lambda x: x.x)
    vehicle_position_gdf['lat'] = vehicle_position_gdf['geometry'].apply(lambda x: x.y)
    vehicle_position_gdf = vehicle_position_gdf[['vehicle_id', 'lon', 'lat']]
    return vehicle_position_gdf

def add_fire(map_vehicles, fire_df):
    fire_polygons = []
    for fire in fire_df.loc[fire_df['lon']<-120].itertuples():
        fire_start_time = getattr(fire, 'start_time')
        fire_end_time = getattr(fire, 'end_time')
        fire_speed = getattr(fire, 'speed')
        fire_type = getattr(fire, 'type')
        fire_origin = getattr(fire, 'geometry')
        fire_offset_dist = getattr(fire, 'offset')
        if fire_start_time > 2000:
            fire_polygons.append([fire_type, fire_origin.buffer(1)])
        elif fire_type != 'initial':
            fire_polygons.append([fire_type, fire_origin.buffer(fire_offset_dist)])
        else:
            fire_time = min(fire_end_time, max(fire_start_time, 2000)) - fire_start_time
            fire_polygons.append([fire_type, fire_origin.buffer(fire_speed * fire_time)])
    fire_polygons_df = pd.DataFrame(fire_polygons, columns=['fire_type', 'geometry'])
    fire_polygons_gdf = gpd.GeoDataFrame(fire_polygons_df, crs='epsg:26910', geometry=fire_polygons_df['geometry']).to_crs('epsg:4326')
    
    map_vehicles.add_data(data=fire_polygons_gdf, name='fire_polygons')
    return map_vehicles

def add_closed_links(closed_link_position):
    closed_link_df = pd.DataFrame(closed_link_position, columns=['link_id', 'x', 'y'])
    closed_link_gdf = gpd.GeoDataFrame(closed_link_df, crs='epsg:26910', geometry=[Point(xy) for xy in zip(closed_link_df['x'], closed_link_df['y'])]).to_crs('epsg:4326')
    closed_link_gdf['lon'] = closed_link_gdf['geometry'].apply(lambda x: x.x)
    closed_link_gdf['lat'] = closed_link_gdf['geometry'].apply(lambda x: x.y)
    closed_link_gdf = closed_link_gdf[['link_id', 'lon', 'lat']]
    return closed_link_gdf

def make_map(network):
    run_vehicle_position, queue_vehicle_position, closed_link_position = extract_vehicle_locations(network)
    # make gdf
    queue_vehicle_position_gdf = make_gdf(queue_vehicle_position)
    run_vehicle_position_gdf = make_gdf(run_vehicle_position)
    closed_link_gdf = add_closed_links(closed_link_position)
    # make map
    with open('keplergl_config.json') as jsonfile:
        map_config = json.load(jsonfile)
    map_vehicles = KeplerGl(height=400, config=map_config)
    # map_vehicles = KeplerGl(height=400)
    map_vehicles.add_data(data=queue_vehicle_position_gdf, name='queue vehicles')
    map_vehicles.add_data(data=run_vehicle_position_gdf, name='run vehicles')
    map_vehicles.add_data(data=closed_link_gdf, name='closed_links')
    return map_vehicles

In [4]:
# preparation
network, links_raster, evacuation_zone, evacuation_buffer, fire_df, check_traffic_flow_links, scen_nm, simulation_outputs = dta_meso.preparation(random_seed=0, dept_time_col='dept_time_scen_1')

fitness=0
for t in range(0, 1):
    step_fitness, network = dta_meso.one_step(t, network, links_raster, evacuation_zone, evacuation_buffer, fire_df, check_traffic_flow_links, scen_nm, simulation_outputs)
    fitness += step_fitness

cawfe_1
log file created
# agents from file  20114


In [5]:
visualization_t_list = {200, 800, 2000, 4000, 6000, 8000, 12000, 14000, 16000, 18000}
for t in range(1, 18001):
    step_fitness, network = dta_meso.one_step(t, network, links_raster, evacuation_zone, evacuation_buffer, fire_df, check_traffic_flow_links, scen_nm, simulation_outputs)
    if step_fitness is not None:
        fitness += step_fitness
    if t in visualization_t_list:
        # visualization_t_dict[t] = make_map(network)
        map = make_map(network)
        map.save_to_html(file_name="projects/butte_osmnx/visualization_outputs/map_{}.html".format(t))

[    0  1740  3042  4242  4329  4369  4680  5535  6046  6192  6207  6225
  6431  7002  7004  8144  8555  8578  8706  8719  9517  9518  9522  9882
  9883  9914  9940 10055 10056 11078 11079 11080 11334 13476 13477 13478
 13522 13528 13529 14173 14378 14379 14419 14420 16256 16289 16290 16364
 16365 16366 16415 16422 16431 16432 16451 16452 19635 19636 19637 19943
 19944 19989 20005 20015 20017 20018 20036 20037 21856 21858 22105 22245
 22339 22652 22743 22910 23824 23938 23939 23940 25594 26202 26356 27173
 27487 27488 27489 27751 27752 28138 28139 28148 28357 28363 28750 29040
 29041 29085 29099 29100 29132 29134 29853 29895 29896 29897 29898 31038
 31039 31070 31071 31082 31083 31088 31089 31090 31244 31245 31247 31275
 31276 31287 31445 32745 32746 32764 32896 32897 32925]
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to projects/butte_osmnx/visualization_outputs/map_800.html!
[    0  1740  3042  4242  4329  4369  4680  5535  6046  6192  6207  6225
  6431  7002  

/home/bingyu/Documents/spatial_queue/model/dta_meso_butte.py:115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (link.link_id in links_in_fire)and (link.burnt == 'not_yet'):


[    0  1740  3042  4242  4310  4329  4369  4383  4680  5535  6046  6192
  6207  6225  6431  7002  7004  8144  8555  8578  8706  8719  9517  9518
  9522  9882  9883  9914  9940 10055 10056 11078 11079 11080 11334 13476
 13477 13478 13522 13528 13529 14172 14173 14378 14379 14419 14420 16256
 16289 16290 16364 16365 16366 16415 16422 16431 16432 16451 16452 19635
 19636 19637 19943 19944 19989 20005 20015 20017 20018 20036 20037 21856
 21858 22105 22245 22339 22631 22652 22743 22910 22922 23824 23938 23939
 23940 25594 26202 26356 27173 27487 27488 27489 27751 27752 28138 28139
 28148 28357 28363 28750 29040 29041 29085 29099 29100 29132 29134 29853
 29895 29896 29897 29898 31038 31039 31070 31071 31082 31083 31088 31089
 31090 31244 31245 31247 31275 31276 31287 31445 32745 32746 32764 32896
 32897 32925]
[    0  1740  3042  4242  4310  4329  4369  4382  4383  4680  5535  6046
  6192  6207  6225  6431  7002  7004  8144  8555  8578  8706  8719  9517
  9518  9522  9882  9883  9914  9940 

/home/bingyu/Documents/spatial_queue/model/dta_meso_butte.py:115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (link.link_id in links_in_fire)and (link.burnt == 'not_yet'):


[    0  1740  2946  3042  4039  4130  4133  4242  4310  4329  4369  4382
  4383  4680  5535  6046  6192  6207  6225  6431  7002  7004  8144  8555
  8578  8706  8719  9517  9518  9522  9882  9883  9914  9940 10055 10056
 10745 11078 11079 11080 11334 13476 13477 13478 13522 13528 13529 14172
 14173 14378 14379 14419 14420 15795 15796 16256 16289 16290 16364 16365
 16366 16415 16422 16431 16432 16451 16452 18118 19635 19636 19637 19943
 19944 19989 20005 20015 20017 20018 20036 20037 20556 20886 20888 21856
 21858 22105 22245 22339 22631 22652 22743 22910 22922 23824 23938 23939
 23940 24357 24372 25594 26139 26146 26202 26356 27173 27449 27487 27488
 27489 27751 27752 28138 28139 28148 28357 28363 28750 28751 28752 29040
 29041 29085 29099 29100 29132 29134 29853 29867 29895 29896 29897 29898
 31038 31039 31070 31071 31082 31083 31088 31089 31090 31244 31245 31247
 31248 31275 31276 31287 31294 31445 32745 32746 32764 32896 32897 32925]
[    0   893  1740  2943  2944  2946  3042  3127  